# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [98]:
import pandas as pd
import pymongo

In [99]:
#connecter avec mongodb
client = pymongo.MongoClient("mongo:27017")
#créer un database "exercices" et un collection "youtube"
database = client['exercices']
collection = database['youtube']

In [100]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [101]:
df_youtube = df_youtube.dropna()

In [102]:
#importer le fichier json à dataframe
import json

with open('./data/US_category_id.json') as data_file:    
    data = json.load(data_file)  

df_categorie = pd.json_normalize(data, 'items')
df_categorie.head(5)

,kind,etag,id,snippet.channelId,snippet.title,snippet.assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [103]:
#nettoyer le dataframe de catégorie
df_categorie=df_categorie.rename(columns={'id':'category_id'})
df_categorie=df_categorie.rename(columns={'snippet.channelId':'snippet_channelId'})
df_categorie=df_categorie.rename(columns={'snippet.title':'snippet_title'})
df_categorie=df_categorie.rename(columns={'snippet.assignable':'snippet_assignable'})
df_categorie["category_id"] = pd.to_numeric(df_categorie["category_id"])
df_categorie.head(5)

,kind,etag,category_id,snippet_channelId,snippet_title,snippet_assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [104]:
df_categorie=df_categorie.drop(['kind'], axis=1)
df_categorie.head(5)

,etag,category_id,snippet_channelId,snippet_title,snippet_assignable
0,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


### Importer les données

In [105]:
#merger le dataframe de catégorie pour récupérer le nom des catégories.
df = pd.merge(df_youtube,df_categorie, on='category_id',how='left')
df=df.drop_duplicates(subset=['video_id'], keep="first")
df.head(5)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,etag,snippet_channelId,snippet_title,snippet_assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...",UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...",UCBR8-60-B28hp2BmDPdntcQ,Comedy,True


In [106]:
#couper les tags en listes
df['tags'] = df['tags'].str.split("|")
df

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,etag,snippet_channelId,snippet_title,snippet_assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
2,cLdxuaxaQwc,My Response,PewDiePie,22,[[none]],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...",UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,"[apple iphone x hands on, Apple iPhone X, iPho...",2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,"[jacksfilms, parody, parodies, iphone, iphone ...",1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...",UCBR8-60-B28hp2BmDPdntcQ,Comedy,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7898,BpGrq3LgbfA,Josie and The Pussycats ft. Cheryl Blossom | M...,BlackStardust,1,[Riverdale - Josie and The Pussycats - CW - Ch...,2205,11,0,0,https://i.ytimg.com/vi/BpGrq3LgbfA/default.jpg,22.10,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
7947,S9VIKOuZcds,My Sweet Jax (Tribute to a Cat),Hot Dad,23,"[cat, feline, pets, beloved, family member, gr...",25037,2846,11,537,https://i.ytimg.com/vi/S9VIKOuZcds/default.jpg,22.10,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...",UCBR8-60-B28hp2BmDPdntcQ,Comedy,True
7949,c74FjEqhCsc,Pistons LIVE Postgame 10.18.17: Stan Van Gundy,FOX SportsDetroit,17,[[none]],1280,11,0,0,https://i.ytimg.com/vi/c74FjEqhCsc/default.jpg,22.10,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",UCBR8-60-B28hp2BmDPdntcQ,Sports,True
7952,a5NIg5yyHWo,Pawn Stars: An Original 1978 Superman Costume ...,HISTORY,24,"[history, history channel, history shows, hist...",400104,2432,123,386,https://i.ytimg.com/vi/a5NIg5yyHWo/default.jpg,22.10,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True


In [107]:
#importer les donnée de fichier USvideos.csv
collection.insert_many(df.to_dict('records'))

## Question 1  

In [108]:
#1) Récupérer toutes les vidéos de la chaîne Apple.
chaine_apple = collection.find( { "channel_title": "Apple" }  )
list(chaine_apple)

[{'_id': ObjectId('61f7234f3f8293cf9c99b29b'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09,
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"',
  'snippet_channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'snippet_title': 'Science & Technology',
  'snippet_assignable': True},
 {'_id': ObjectId('61f7234f3f8293cf9c99b452'),
  'video_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'airpods',
   'apple watch

## Question 2

In [109]:
#2) Compter le nombre de catégories différentes
categorie=collection.distinct('snippet_title')
print("le list de catégories différentesest :", categorie)
print("le nombre de catégories différentesest :", len(categorie))

le list de catégories différentesest : ['Autos & Vehicles', 'Comedy', 'Education', 'Entertainment', 'Film & Animation', 'Gaming', 'Howto & Style', 'Music', 'News & Politics', 'Nonprofits & Activism', 'People & Blogs', 'Pets & Animals', 'Science & Technology', 'Shows', 'Sports', 'Travel & Events']
le nombre de catégories différentesest : 16


## Question 3

In [111]:
#3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
print("tags est déjà coupé en listes")

tags est déjà coupé en listes


## Question 4

In [112]:
#4) Récupérer les vidéos les plus vues.
view_max=collection.find().sort([("views", -1)]).limit(20)
print("les 20 vidéos les plus vues : ")
list(view_max)

les 20 vidéos les plus vues : 


[{'_id': ObjectId('61f7234f3f8293cf9c99b34b'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'],
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': 13.09,
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
  'snippet_channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'snippet_title': 'Music',
  'snippet_assignable': True},
 {'_id': ObjectId('61f7234f3f8293cf9c99b45e'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 20565649,
  'likes': 1478119,
  'dislikes': 46477,
  'comment_total': 501435,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  '

## Question 5

In [113]:
#5) Compter le nombre moyen de vues en fonction de la catégorie.
Moy_views=collection.aggregate([{'$group':{'_id':'$snippet_title','Moyenne de vues':{'$avg':"$views"}}}])
list(Moy_views)

[{'_id': 'Shows', 'Moyenne de vues': 8259.0},
 {'_id': 'Science & Technology', 'Moyenne de vues': 515854.13907284767},
 {'_id': 'News & Politics', 'Moyenne de vues': 374883.0505050505},
 {'_id': 'Pets & Animals', 'Moyenne de vues': 412381.6216216216},
 {'_id': 'Sports', 'Moyenne de vues': 470686.1677852349},
 {'_id': 'Comedy', 'Moyenne de vues': 828485.2676056338},
 {'_id': 'Nonprofits & Activism', 'Moyenne de vues': 572023.2},
 {'_id': 'Education', 'Moyenne de vues': 368229.0425531915},
 {'_id': 'Gaming', 'Moyenne de vues': 492390.2413793103},
 {'_id': 'Autos & Vehicles', 'Moyenne de vues': 415129.0540540541},
 {'_id': 'Film & Animation', 'Moyenne de vues': 558289.1881188119},
 {'_id': 'Music', 'Moyenne de vues': 697598.5910447761},
 {'_id': 'Travel & Events', 'Moyenne de vues': 300664.1111111111},
 {'_id': 'People & Blogs', 'Moyenne de vues': 597891.1789883268},
 {'_id': 'Entertainment', 'Moyenne de vues': 555071.1740041929},
 {'_id': 'Howto & Style', 'Moyenne de vues': 405748.290076

## Question 6 

In [114]:
#6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.
Moy_likes=collection.aggregate([{'$group':{'_id':'$channel_title','Moyenne de likes':{'$avg':"$likes"}}},{"$sort": {"Moyenne de likes": -1}},{"$limit": 20}])
print("Les 20 chaines Youtube avec la plus grande moyenne de likes sont :")
list(Moy_likes)

Les 20 chaines Youtube avec la plus grande moyenne de likes sont :


[{'_id': 'ZaynVEVO', 'Moyenne de likes': 1431683.0},
 {'_id': 'ibighit', 'Moyenne de likes': 1112071.0},
 {'_id': 'melanie martinez', 'Moyenne de likes': 902166.0},
 {'_id': 'jypentertainment', 'Moyenne de likes': 721796.0},
 {'_id': 'Logan Paul Vlogs', 'Moyenne de likes': 601533.5},
 {'_id': 'Wengie', 'Moyenne de likes': 455904.0},
 {'_id': 'Conor Maynard', 'Moyenne de likes': 389273.0},
 {'_id': 'Dude Perfect', 'Moyenne de likes': 363320.0},
 {'_id': 'PewDiePie', 'Moyenne de likes': 351994.0},
 {'_id': 'Primitive Technology', 'Moyenne de likes': 328200.0},
 {'_id': 'Gabbie Hanna', 'Moyenne de likes': 315097.0},
 {'_id': 'Alesso', 'Moyenne de likes': 306843.0},
 {'_id': 'Ed Sheeran', 'Moyenne de likes': 306217.0},
 {'_id': 'Liza Koshy', 'Moyenne de likes': 306043.2},
 {'_id': 'FifthHarmonyVEVO', 'Moyenne de likes': 291383.0},
 {'_id': 'Daniel Howell', 'Moyenne de likes': 271091.0},
 {'_id': 'ImagineDragonsVEVO', 'Moyenne de likes': 259123.0},
 {'_id': 'Bhad Bhabie', 'Moyenne de likes'